<a href="https://colab.research.google.com/github/RJuro/Africalics-PhD-Academy-2018/blob/master/notebooks/SDS_M1___S1_2___Exercise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Installing pyreadr to be able to work with R-data formats
!pip install pyreadr

     |████████████████████████████████| 225kB 2.7MB/s 


## Stuff to do

- Remove missing data (where that is needed/useful) - Columns/Rows
- Recode variable types (objects to boolian and other where Pandas did not detect automatically )
- Create Date-Time index
- Think about some interesting things to explore!

In [2]:
# Download the data
!wget http://sds-datacrunch.aau.dk/public/SDS2019/new_orleans_2019.rds

--2019-09-05 07:37:08--  http://sds-datacrunch.aau.dk/public/SDS2019/new_orleans_2019.rds
Resolving sds-datacrunch.aau.dk (sds-datacrunch.aau.dk)... 192.38.56.127
Connecting to sds-datacrunch.aau.dk (sds-datacrunch.aau.dk)|192.38.56.127|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12197131 (12M) [application/octet-stream]
Saving to: ‘new_orleans_2019.rds’

new_orleans_2019.rd 100%[===================>]  11.63M  7.73MB/s    in 1.5s    

2019-09-05 07:37:09 (7.73 MB/s) - ‘new_orleans_2019.rds’ saved [12197131/12197131]



In [0]:
# Import pandas and pyreadr
import pyreadr
import pandas as pd

In [0]:
# Read in the downloaded file as described in pyreadr-documentation https://github.com/ofajardo/pyreadr
result = pyreadr.read_r('new_orleans_2019.rds')
data = result[None]

In [14]:
#quick check of the data
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 513065 entries, 0 to 513064
Data columns (total 30 columns):
raw_row_number        513065 non-null object
date                  513055 non-null object
time                  513065 non-null float64
location              416421 non-null object
lat                   260624 non-null float64
lng                   260624 non-null float64
district              513065 non-null object
zone                  513065 non-null object
subject_age           499880 non-null float64
subject_race          500938 non-null category
subject_sex           500938 non-null category
officer_assignment    512941 non-null object
type                  362916 non-null category
arrest_made           389716 non-null object
citation_issued       389716 non-null object
warning_issued        389716 non-null object
outcome               335680 non-null category
contraband_found      75802 non-null object
contraband_drugs      75802 non-null object
contraband_weapons    75

In [6]:
# Check for missing data - especially date and time
data.isnull().sum()

raw_row_number             0
date                      10
time                       0
location               96644
lat                   252441
lng                   252441
district                   0
zone                       0
subject_age            13185
subject_race           12127
subject_sex            12127
officer_assignment       124
type                  150149
arrest_made           123349
citation_issued       123349
warning_issued        123349
outcome               177385
contraband_found      437263
contraband_drugs      437263
contraband_weapons    437263
frisk_performed       123349
search_conducted      123349
search_person         123349
search_vehicle        123349
search_basis          437263
reason_for_stop            0
vehicle_color         239498
vehicle_make          236111
vehicle_model         253379
vehicle_year          240697
dtype: int64

In [0]:
# Drop rows with no date to avoid problems when we try to transform to timestamps
data.dropna(subset=['date'], inplace=True)

As you could see, the time-column is read in as a float (floating point real value − that represents a real numbers and is written with a decimal point) of the number of seconds in the point of time of the day (thanks Claus and Frederick for solving that)

To transform that into a timestamp we will need the pandas' timedelta feature - since the seconds represent a timespan, that should work

In [0]:
# apply timedelta
data['time'] = pd.to_timedelta(data['time'], unit='seconds')

In [29]:
data['time'][0]

Timedelta('0 days 01:11:00')

In [0]:
# transform dates to date-timestamps
data['date'] = pd.to_datetime(data['date'])

In [0]:
# now we can construct our timestamps by simply adding the timedeltas (0 days and some time) to our timestamps from dates
data['timestamp'] = data['time'] + data['date']

In [0]:
# set index as our timestamp
data.set_index('timestamp', inplace=True)

In [39]:
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 513065 entries, 2010-01-01 01:11:00 to 2018-07-18 00:34:00
Data columns (total 30 columns):
raw_row_number        513065 non-null object
date                  513055 non-null datetime64[ns]
time                  513065 non-null timedelta64[ns]
location              416421 non-null object
lat                   260624 non-null float64
lng                   260624 non-null float64
district              513065 non-null object
zone                  513065 non-null object
subject_age           499880 non-null float64
subject_race          500938 non-null category
subject_sex           500938 non-null category
officer_assignment    512941 non-null object
type                  362916 non-null category
arrest_made           389716 non-null object
citation_issued       389716 non-null object
warning_issued        389716 non-null object
outcome               335680 non-null category
contraband_found      75802 non-null object
contraband_drugs  

Many of the columns that have a True/False format have many missing values. If the value is missing, it is reasonable to asusme that it is False. 
e.g. if no arrest has been recorded then probably no arrest happened.
Therefore we can fill the missing data with the value False
Also we transform the to boolian datatype

In [0]:
data.frisk_performed = data.frisk_performed.fillna(False)
data.arrest_made = data.arrest_made.fillna(False)

In [0]:
data['frisk_performed'] = data['frisk_performed'].astype(bool)
data['arrest_made'] = data['arrest_made'].astype(bool)

In [65]:
# Now we can resample by time
data.resample('Y')['frisk_performed'].mean()

# you can see that up to ~2010 this variable is likely not properly recorded during police stops in this area.

timestamp
2001-12-31    0.000000
2002-12-31    0.000000
2003-12-31    0.000000
2004-12-31    0.000000
2005-12-31         NaN
2006-12-31         NaN
2007-12-31    0.000000
2008-12-31    0.003968
2009-12-31    0.000000
2010-12-31    0.000048
2011-12-31    0.063446
2012-12-31    0.121595
2013-12-31    0.154668
2014-12-31    0.140057
2015-12-31    0.144756
2016-12-31    0.194375
2017-12-31    0.216618
2018-12-31    0.152336
Name: frisk_performed, dtype: float64

Do men and women have different vehicles colors?

In [0]:
pd.crosstab(data.vehicle_color,data.subject_sex, normalize='index')

In [0]:
pd.crosstab(data.vehicle_make,data.search_vehicle, normalize='index')